## General trash dataset 폴더 생성

In [10]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import display
import numpy as np

In [6]:

# 기본 경로 및 폴더 이름 설정
dataset_path = '/data/ephemeral/home/'
dataset_name = 'general_trash_dataset'

general_trash_images = 'general_images'

os.makedirs(os.path.join(dataset_path, dataset_name, general_trash_images), exist_ok=True)

## General Trash bbox crop

In [7]:

# train.json 파일 경로
train_path = '/data/ephemeral/home/dataset/train.json'

save_folder = '/data/ephemeral/home/general_trash_dataset/general_images'

# json 파일 로드
with open(train_path, 'r') as file :
    data = json.load(file)

# general_trash annotation 정보 필터링
general_sort = [anno for anno in data['annotations'] if anno['category_id'] == 0]

# images 데이터 id를 키로 하는 딕셔너리 변환
images_dict = {image['id']: image for image in data['images']}

# annotation에 대해 image crop
for anno in general_sort :
    image_data = images_dict[anno['image_id']]
    image_path = '/data/ephemeral/home/dataset/' + image_data['file_name']

    with Image.open(image_path) as img :
        bbox = anno['bbox']
        if bbox[2] <= 0 or bbox[3] <= 0:
            print(f"Invalid bbox size for annotation {anno['id']}. Skipping.")
            continue

        try :
            cropped_img = img.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
            cropped_img_path = os.path.join(save_folder, f"{anno['image_id']}_{anno['id']}.jpg")
            cropped_img.save(cropped_img_path)

        except Exception as e :
            print(f"Error cropping image for annotation {anno['id']} : {e}")

Error cropping image for annotation 5159 : cannot write empty image as JPEG


## crop된 data 갯수 확인

In [8]:
folder_path = '/data/ephemeral/home/general_trash_dataset/general_images'

image_extensions = '.jpg'

image_count = sum(1 for file in os.listdir(folder_path) if file.endswith(image_extensions))

print(f"Number of image files in '{folder_path}' : {image_count}")

# 5159번 bounding box 이미지 제외하고 전부 crop 후 저장 확인 완료
# 5159번 이미지 바운딩박스가 거의 점처럼 찍혀있음, labeling 잘못 된 객체

Number of image files in '/data/ephemeral/home/general_trash_dataset/general_images' : 3965


## crop 된 Image의 평균 width, height 계산

In [14]:
# 이미지 너비와 높이 데이터 저장을 위한 리스트
widths = []
heights = []

# 폴더 내의 모든 이미지 파일에 대해 너비와 높이 계산
for file_name in os.listdir(folder_path):
    if file_name.endswith('.jpg'):
        image_path = os.path.join(folder_path, file_name)
        with Image.open(image_path) as img:
            width, height = img.size
            widths.append(width)
            heights.append(height)

# 너비와 높이를 10개의 범주로 나누기
num_bins = 10
width_bins = np.linspace(min(widths), max(widths), num_bins)
height_bins = np.linspace(min(heights), max(heights), num_bins)

# 각 범주 내에서 너비와 높이의 평균 계산
average_widths = [np.mean([w for w in widths if w >= width_bins[i] and w < width_bins[i+1]]) for i in range(len(width_bins)-1)]
average_heights = [np.mean([h for h in heights if h >= height_bins[i] and h < height_bins[i+1]]) for i in range(len(height_bins)-1)]

# 가장 일반적인 너비와 높이 조합 찾기
most_common_width = max(set(average_widths), key=average_widths.count)
most_common_height = max(set(average_heights), key=average_heights.count)

print(f"The most common average image size is approximately: {most_common_width:.2f} x {most_common_height:.2f}")

The most common average image size is approximately: 67.31 x 513.13


## 평균 width x height로 이미지 리사이징

In [16]:
def resize_image_with_aspect_ratio(image, output_size, background_color=(0, 0, 0)):
    # 원본 이미지의 비율 계산
    original_width, original_height = image.size
    ratio = min(output_size[0] / original_width, output_size[1] / original_height)

    # 새로운 사이즈에 맞게 이미지 리사이징
    new_size = (int(original_width * ratio), int(original_height * ratio))
    resized_img = image.resize(new_size, Image.Resampling.LANCZOS)

    # 새 이미지 생성 및 원본 이미지 붙여넣기
    new_img = Image.new("RGB", output_size, background_color)
    new_img.paste(resized_img, ((output_size[0] - new_size[0]) // 2, (output_size[1] - new_size[1]) // 2))

    return new_img

# 크롭된 이미지가 저장된 폴더 경로
crop_folder_path = '/data/ephemeral/home/general_trash_dataset/general_images'

# 리사이징된 이미지를 저장할 폴더 경로
resize_folder_path = '/data/ephemeral/home/general_trash_dataset/general_trash_resize'

# 리사이징할 사이즈 (67 x 513)
output_size = (67, 513)

# 리사이징된 이미지 저장 폴더 생성
if not os.path.exists(resize_folder_path):
    os.makedirs(resize_folder_path)

# 폴더 내의 모든 이미지 파일에 대해 리사이징 수행
for file_name in os.listdir(crop_folder_path):
    if file_name.endswith('.jpg'):
        image_path = os.path.join(crop_folder_path, file_name)
        with Image.open(image_path) as img:
            resized_image = resize_image_with_aspect_ratio(img, output_size)
            resized_image.save(os.path.join(resize_folder_path, file_name))
